In [ ]:
import numpy as np
import ilmsens_hal
import ilmsens_hal.utils
import matplotlib.pyplot as plt
from ilmsens_hal.defn import meas_run
from ilmsens_hal.types import ilmsens_hal_ModConfig
from ilmsens_hal.defn import meas_config
from ilmsens_hal.defn import config
from ilmsens_hal.defn import diag
import matplotlib.pyplot as plt


Print HAL version

In [ ]:

ver = ilmsens_hal.getVersion()
print(f"HAL Version: {ver.mMajor}.{ver.mMinor}.{ver.mBuild}")

Initialize the HAL and set the debugging level

In [ ]:

_ = ilmsens_hal.setDEBLevel(diag.ILMSENS_DEB_NO)
num_devices = ilmsens_hal.initHAL()
connected_devices = list(range(1, num_devices+1))
print(f"Found {num_devices} connected devices")

Get the unique device identifier of each connected sensor

In [ ]:

for idx in connected_devices:
    device_id = ilmsens_hal.getModId(idx)
    print(f"Sensor #{idx} has ID '{device_id.decode()}' (result was {len(device_id)}).")

Select and open the first sensor and get the sensor unique device identifier

In [ ]:

selected_device_idx = 1
_ = ilmsens_hal.openSensors([selected_device_idx])

Set the sensor's configuration using the `ilmsens_hal_ModConfig` object

In [ ]:

mod_config = ilmsens_hal_ModConfig()
mod_config.mClk = 13.312
mod_config.mOV = 0 # 0 = use default
mod_config.mOrder = 9
mod_config.mRx = 2
mod_config.mSub = 0 # 0 = use default
mod_config.mTx = 0 # 0 = use default

swavg = 32 # num of software averages
wait_cyc = 0

ilmsens_hal.setupSensors([selected_device_idx], mod_config)

# set the sensor as Master
ilmsens_hal.setMaster([selected_device_idx], config.ILMSENS_HAL_MASTER_SENSOR)

# restart the digital synchronization
ilmsens_hal.synchMS([selected_device_idx], meas_config.ILMSENS_HAL_SYNCH_OFF)
ilmsens_hal.synchMS([selected_device_idx], meas_config.ILMSENS_HAL_SYNCH_ON)

# reset the M-sequence generator (transmitter)
ilmsens_hal.setMLBS([selected_device_idx])

# set the software averages and wait cycles (0 means snapshot mode)
ilmsens_hal.setAvg([selected_device_idx], swavg, wait_cyc)

Show the sensor's information and configuration to confirm

In [ ]:

info = ilmsens_hal.getModInfo(selected_device_idx)
print("Configuration of selected sensor is :")
print("* RF system clock    [GHz]: {:.6f}".format(info.mConfig.mClk))
print("* MLBS order              : {}".format(info.mConfig.mOrder))
print("* Prescaler           1/  : {}".format(info.mConfig.mSub))
print("* Oversampling        x   : {}".format(info.mConfig.mOV))
print("* Number of Tx            : {}".format(info.mConfig.mTx))
print("* Number of Rx            : {}".format(info.mConfig.mRx))
print("* Number of samples per Rx: {}".format(info.mNumSamp))
print("* Hardware averages       : {}".format(info.mHWAvg))
print("* Software avg. limits    : [{} .. {}]".format(info.mAvgLim[0], info.mAvgLim[1]))
print("* Software averages       : {}".format(info.mAvg))
print("* Wait cycle limits       : [{} .. {}]".format(info.mWaitLim[0], info.mWaitLim[1]))
print("* Wait cycles             : {}".format(info.mWait))
print("* ADC full scale range [V]: [{:.6f} .. {:.6f}]".format(info.mFSR[0], info.mFSR[1]))
print("* ADC LSB voltage     [mV]: {:.6f}".format(info.mLSB_Volt*1000.0))
print("* Int. temperature    [\370C]: {:.2f}".format(info.mTemp))

Turn on the transmitter

In [ ]:

ilmsens_hal.setPD([selected_device_idx], meas_config.ILMSENS_HAL_TX_ON)

Run measurement, wait for the data transfer, then stop the measurement.  
Get the output data and process it using the `ilmsens_hal.utils` module.

In [ ]:

ilmsens_hal.measRun([selected_device_idx], meas_run.ILMSENS_HAL_RUN_BUF)

# wait for the data transfer
data, _ = ilmsens_hal.measGet([selected_device_idx], timeout_millis=100)

ilmsens_hal.measStop([selected_device_idx])

data = ilmsens_hal.utils.parse_data(data)
data = np.array(data['rx1_samples'], dtype=np.double)

# scale data to represent physical voltage
averages = info.mHWAvg * swavg
value_scale = 1.0 / averages * info.mLSB_Volt
off_value = 0.0
data = (data - off_value) * value_scale

# get the ideal sequence, time axis, and frequency axis.
dependencies = ilmsens_hal.utils.read_dependencies(mDR_MLBS_Order=9, mDR_F0_Clk=13.312, mDR_OV=1)

# correlate the data with the ideal sequence
data = np.fft.fft(data, axis=0)
ideal_ref_spec = dependencies['mDR_Ref_Spec']
data = data * ideal_ref_spec[:]
data = np.fft.ifft(data, axis=0)

# plot the processed impulse response
plt.figure(figsize=(25, 5))
plt.plot(data)
plt.grid(which='both')
plt.minorticks_on()
plt.ylim([-1, 1])
plt.xlim([0, 511])

Close sensors and deinitialize the HAL

In [ ]:

ilmsens_hal.closeSensors(connected_devices)
ilmsens_hal.deinitHAL()